In [215]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import sys
import time
import feather
import numpy as np
import pandas as pd
from contextlib import contextmanager
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import lightgbm as lgbm
import optuna

sys.path.append("/Users/ueda/Desktop/kaggle/mynavi/code/src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage

In [216]:
# ===============
# Constants
# ===============
DATA_DIR = "../input/"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH = os.path.join(DATA_DIR, "test.csv")

SUB_PATH = os.path.join(DATA_DIR, "sample_submit.csv")
#FOLDS_PATH = "../input/mynavi_Stratifiedfold01.feather"   #289381102       #folds気をつけて
#FOLDS_PATH = "../input/mynavi_Stratifiedfold_access.feather"  #有望

FOLDS_PATH = "../input/mynavi_Stratifiedfold_address.feather" #274592066 最強
#FOLDS_PATH = "../input/mynavi_Stratifiedfold_rent.feather"     #299584731
#FOLDS_PATH = "../input/mynavi_Stratifiedfold_same_build.feather" # 293669458
LOGGER_PATH = "log.txt"
ID_COLUMN = "id"


EXP_ID = "exp0_+address_poisson_outlin+meta"

SAVE_PATH = f'../output/{EXP_ID}'
try:
    os.mkdir(SAVE_PATH)
except:
    print("Already directory created")

Already directory created


In [312]:
# ===============
# Settings
# ===============
pd.set_option('display.max_columns', 50)

SEED = 0
N_SPLITS = 5
LOG = False
LGBM_PARAMS = {
    'task': 'train',
    'boosting_type': 'gbdt',   #rf, gbdt
    'objective': 'poisson',    #regression, gamma, poisson gamma早すぎ
    'metric': "rmse",
    'learning_rate': 0.2,
    'max_depth': 8,            #9
    'num_leaves': int(.7 * 8 ** 2),        #15 < int()
    'subsample': 0.9,
    'subsample_freq': 1,
    'colsample_bytree': 0.8, # 0.4, 0.7, 0.8で271219355 0.8良さげ
    'max_bin': 255,
    'reg_alpha': 0.1,        # 0.1
    'reg_lambda': 0.1,
    'min_child_samples': 20,   #20 vs 10
    'min_gain_to_split': 0.02,
    'min_data_in_bin': 3,
    'bin_construct_sample_cnt': 5000,
    'cat_l2': 5,    #5でも10でも
    'verbose': -1,
    'nthread': -1,
    'seed': SEED,
}
LGBM_FIT_PARAMS = {
    'num_boost_round': 50000,
    'early_stopping_rounds': 100,
    'verbose_eval': 5000,
}

use_cols = []

drop_columns = ['fold_id', 'id', 'キッチン', 'バス・トイレ', '周辺環境',
                '室内設備', '所在階', '放送・通信', '間取り',
                '築年数', '賃料', '面積', '駐車場', "路線_0", "駅_0", "方角", #"所在地", #"same_build",
                 "路線_1", "駅_1", "路線_2", "駅_2", "契約期間", '建物構造', 'アクセス', #"地域_n丁目", "地域名",# "catuse_駅_0", "catuse_路線_0"
]

#ategorical_features = ["catuse_駅_0", "catuse_路線_0", #"地域名",
#                        "cat_int間取り", "cat_building_height",'所在_区'
#]

categorical_features = ["catuse_駅_0", "catuse_路線_0", "所在地", "地域名", "地域_n丁目",
                        "cat_int間取り", "cat_building_height",'所在_区',
] #258442952

#TE_columns = ["所在_区", "間取りtype", 'catuse_路線_0', 'catuse_駅_0',
#           "cat_int間取り", "cat_building_height", "地域名", '地域_n丁目', "same_build"
#]
#drop_columns = drop_columns + [col for col in categorical_features if col in TE_columns]
#categorical_features = [cat_col for cat_col in categorical_features if cat_col not in TE_columns]

FEATURES = [
    "../code/feature_csv/base_feature1.feather",
    "../code/feature_csv/nonleak_group_feature2.feather",
    #'../code/feature_csv/nonleak_nearStation_group_feature.feather',
    #'../code/feature_csv/near_access_feature.feather',
    "../code/feature_csv/word_contain_sparse_feature.feather",
    "../code/feature_csv/access_feature.feather",
    "../code/feature_csv/address_latlng.feather",
    #"../code/feature_csv/meta_features.feather"
]
setup_logger(out_file=LOGGER_PATH)

2019-11-06 20:30:45,798 - INFO - logger set up


<RootLogger root (DEBUG)>

In [313]:
# ===============
# Functions
# ===============
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
def load_feature_feather(df, filename):
    feature = feather.read_dataframe(filename)
    df = pd.concat((df, feature), axis=1)
    del feature
    gc.collect()
    return df

def load_feature_csv(df, filename):
    feature = pd.csv(filename)
    df = pd.concat((df, feature), axis=1)
    del feature
    gc.collect()
    return df

def modify_rent(target, df_input):
    """ id == 5776 で賃料の0が一つ多い"""
    target = np.where(df_input["id"]==5776, 120350, target)
    
    return target


"""def TE_feature(df_main, df_add, TE_columns = TE_columns):
    df_out = pd.DataFrame()
    add_TE_cols = [f"TE_{col}_mul_面積" for col in TE_columns]
    tmp_cols = [f"TE_{col}" for col in TE_columns]
    TE_df = df_add[TE_columns]
    TE_df.columns = tmp_cols
    
    TE_mul_df = pd.DataFrame(TE_df[tmp_cols].values * df_main["float面積"].values.reshape(-1,1), columns=add_TE_cols)
    TE_mul_df.index = df_main.index
    TE_df.index = df_main.index
    
    TE_mul_df["std_TE賃料"] = TE_mul_df.std(axis=1)
    TE_mul_df["min_TE賃料"] = TE_mul_df.min(axis=1)
    TE_mul_df["max_TE賃料"] = TE_mul_df.max(axis=1)
    TE_mul_df["mean_TE賃料"] = TE_mul_df.mean(axis=1)
    
    df_out = pd.concat([TE_df, TE_mul_df], axis=1)
    
    return df_out"""

def calc_loss(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

## main

In [314]:
with timer("load data"):
    train = feather.read_dataframe(FOLDS_PATH)
    test = pd.read_csv(TEST_PATH)
    if LOG:
        y = np.log1p(train['賃料'].copy())
    else:
        y = train['賃料'].copy()
    folds = train[["fold_id"]].copy()
    n_train = len(train)
    train = train.append(test).reset_index(drop=True)
    del test
    gc.collect()

2019-11-06 20:30:47,146 - INFO - [load data] done in 1 s


In [315]:
with timer("load featuers"):
    for f in FEATURES:
        train = load_feature_feather(train, f)
        
    #train["same_build"] = train["building_floor"].astype(str) + train["地域_n丁目"] + train["int築年"].astype(str)

2019-11-06 20:30:51,114 - INFO - [load featuers] done in 4 s


In [316]:
with timer("preprocessing"):
    for c in categorical_features:
        lbl = LabelEncoder()
        lbl.fit(list(train[c].astype("str").values))
        train[c] = lbl.transform(list(train[c].astype("str").values))

    
    """記入ミスの修正"""
    if "id" in train.columns:
        y = modify_rent(y, train[:n_train])
        print("Modifify Done")
        print("****************")
        
    
    
    try:
        train.drop(drop_columns, axis=1, inplace=True)
        print("DROP DONE!")
        print("****************")
    except:
        pass
    
    
    """importtanceで区切るなら"""
    if len(use_cols)==0:
        test = train[n_train:]
        train = train[:n_train]
    
    elif len(use_cols)>0:
        test = test[use_cols]
        train = train[use_cols]
        
    try:
        train.drop(drop_columns, axis=1, inplace=True)
    except:
        pass
    
    
    """外れ値分類の特量追加"""
    out = np.percentile(y, [85])
    outlin =  np.where(y > out,1,0)
    model = lgbm.LGBMClassifier().fit(train, outlin)
    train["out"] = outlin
    test["out"] = model.predict(test)
    
    
    train = reduce_mem_usage(train)
    train.to_feather(f"../features/{EXP_ID}_df.feather")
        
    #gc.collect()
    #target_test = pd.read_csv(f'../code/feature_csv/target_encoding_test.csv')
    #test = pd.concat([test, TE_feature(test, target_test)], axis=1)
    
    features = list(test.columns.values)

Modifify Done
****************
DROP DONE!
****************
Memory usage of dataframe is 230.25 MB
column =  959
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
Memory usage after optimization is: 96.01 MB
Decreased by 58.3%


2019-11-06 20:31:37,348 - INFO - [preprocessing] done in 46 s


In [317]:
"""train.shape
old_df = feather.read_dataframe("../features/exp0_+address_poisson_outlin_df.feather")
[col for col in test.columns if col not in old_df.columns]
#[col for col in old_df.columns if col not in test.columns]
ymp = [col for col in test.columns if col in old_df.columns]

for col in ymp:
    if (train[col]==old_df[col]).nunique() == 2:
        if (np.isnan(train[col]).values == (train[col] != old_df[col]).values).sum() != train.shape[0]:
            print((train[col] == old_df[col]).value_counts())"""

'train.shape\nold_df = feather.read_dataframe("../features/exp0_+address_poisson_outlin_df.feather")\n[col for col in test.columns if col not in old_df.columns]\n#[col for col in old_df.columns if col not in test.columns]\nymp = [col for col in test.columns if col in old_df.columns]\n\nfor col in ymp:\n    if (train[col]==old_df[col]).nunique() == 2:\n        if (np.isnan(train[col]).values == (train[col] != old_df[col]).values).sum() != train.shape[0]:\n            print((train[col] == old_df[col]).value_counts())'

In [ ]:
with timer("train"):
    y_oof = np.empty(len(train), )
    y_test = []
    feature_importances = pd.DataFrame()

    for fold_id in range(N_SPLITS):
        with timer(f"fold {fold_id}"):
            x_train, y_train = train[folds.fold_id != fold_id], y[folds.fold_id != fold_id]
            x_val, y_val = train[folds.fold_id == fold_id], y[folds.fold_id == fold_id]
            
            #TE_fold = pd.read_csv(f'../code/feature_csv/target_encoding_groupfold{fold_id}.csv')
            #x_train = pd.concat([x_train, TE_feature(x_train, TE_fold[TE_fold["fold_id"]!=fold_id])], axis=1)
            #x_val = pd.concat([x_val, TE_feature(x_val, TE_fold[TE_fold["fold_id"]==fold_id])], axis=1)
            
            y_pred_valid, y_pred_test, train_loss, valid_loss, importances, best_iter = train_lgbm(
                x_train, y_train, x_val, y_val, test,
                categorical_features=categorical_features,
                feature_name=features,
                fold_id=fold_id,
                lgb_params=LGBM_PARAMS,
                fit_params=LGBM_FIT_PARAMS,
                model_name=EXP_ID,
                loss_func=calc_loss,
                rank=False,
                calc_importances=True
            )
            y_oof[folds.fold_id == fold_id] = y_pred_valid
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    feature_importances.to_csv(os.path.join(SAVE_PATH, "feature_importances.csv"), index=False)
    if LOG:
        y = np.expm1(y)
        y_oof = np.expm1(y_oof)
        y_test = np.expm1(y_test)
    score = calc_loss(y, y_oof)
    y_test = np.mean(y_test, axis=0)
    yoof_and_target = pd.DataFrame()
    yoof_and_target["賃料"] = y
    yoof_and_target[f'{EXP_ID}'] = y_oof
    yoof_and_target.to_csv(os.path.join(SAVE_PATH, "yoof_and_target.csv"), index=False)
    yoof_and_target.to_csv(f'../output/vs_trains/{EXP_ID}.csv', index=False)
    
    test_result = pd.DataFrame()
    test_result[f"{EXP_ID}"] = y_test
    test_result.to_csv(f'../output/test_results/{EXP_ID}.csv', index=False)
    
    LOGGER.info(f'CV={score}')

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[290]	valid's rmse: 16347.6


2019-11-06 20:31:50,916 - INFO - Best Iteration: 290
2019-11-06 20:31:53,197 - INFO - [fold 0] done in 16 s


Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1009]	valid's rmse: 14764.6


2019-11-06 20:32:50,431 - INFO - Best Iteration: 1009
2019-11-06 20:32:53,855 - INFO - [fold 1] done in 61 s


Training until validation scores don't improve for 100 rounds.


In [ ]:
"""if fold_id does not drop """
#(x_train[x_train["fold_id"]!=0]["fold_id"] == TE_feature[TE_feature["fold_id"]!=0]["fold_id"]).value_counts()

In [ ]:
#TE_feature[TE_columns]["所在_区"].value_counts()

In [ ]:
with timer("sub"):
    sub = pd.read_csv(SUB_PATH, header=None)
    sub[1] = y_test
    LOGGER.info(f'len sub={len(sub)}')
    sub.to_csv(os.path.join(SAVE_PATH,f'{EXP_ID}.csv'), index=False, header=None)

In [ ]:
"""importtances visualization"""
cols = importances[["feature", "gain"]].groupby("feature").mean().sort_values(
    by="gain", ascending=False)[:1000].index

best_features = importances.loc[importances.feature.isin(cols)]
plt.figure(figsize=(20,20))
sns.barplot(x="gain", y="feature", data=best_features.sort_values(by="gain", ascending=False))
plt.title(f'LightGBM Features (avg over folds)_{EXP_ID}')
plt.tight_layout()
plt.savefig(os.path.join(SAVE_PATH,(f'lgbm_importances_{EXP_ID}.png')))

In [ ]:
categorical_features
#TE_feature[TE_columns]

In [ ]:
pd.set_option('display.max_rows', 500)
importances.sort_values("gain", ascending=False)

In [ ]:
use_cols = list(importances[importances["gain"]>30000]["feature"].values)
print(len(use_cols), len(train.columns))
use_cols = [col for col in use_cols if col in train.columns]
use_cols = use_cols + [col for col in categorical_features if col not in use_cols]

In [ ]:
yoof_and_target[yoof_and_target["賃料"] == yoof_and_target["賃料"].max()]

# ENSEMBLE

In [ ]:
x_train[x_train["max_TE賃料"]==x_train["max_TE賃料"].min()]

In [ ]:
test2 = test.reset_index()
[test2["out"]==1]

In [ ]:
sub

In [ ]:
up_model = pd.read_csv("../output/exp1_poisson_upper100000/exp1_poisson_upper100000_100000.csv", header=None)

In [ ]:
ens = sub[[0]]
ens["1"] = (np.where([test2["out"]==1], up_model[1], sub[1])).reshape(-1,1)

In [ ]:
ens.to_csv('ens.csv', index=False, header=None)